In [1]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from keras.layers.convolutional import Conv2D
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import codecs
from tensorflow.contrib.rnn import *

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

In [4]:
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
#vad = np.load('../../data/y_egemaps.npy')
# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

# load model
EMBEDDING_DIM = 300
file_loc = '../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

# split train/test
split = 7869
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)


Found 3437 unique tokens
../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 128


In [5]:
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(10039, 200, 189, 1)

In [6]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [7]:
split = 7869

In [8]:
# API model
def model():
    # speech network
#     input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
#     net_speech = BatchNormalization()(input_speech)
#     net_speech = LSTM(32, return_sequences=True)(net_speech)
#     net_speech = LSTM(64, return_sequences=True)(net_speech)
#     net_speech = LSTM(128, return_sequences=True)(net_speech)
#     net_speech = LSTM(64, return_sequences=True)(net_speech)
#     net_speech = LSTM(32, return_sequences=True)(net_speech)
#     model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)
    
    #mocap network
    input_mocap = Input(shape=(200,189,1))
    model_mocap0 = BatchNormalization()(input_mocap)
    model_mocap1 = Conv2D(64, 3, strides=(3, 3), padding="same")(model_mocap0)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Conv2D(256, 3,strides=(3, 3), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Conv2D(64, 3,strides=(3, 3), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    #model_mocap = Dropout(dropout_rate)(model_mocap16)
    # combined model
    model_combined = concatenate([model_text, model_mocap])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_text, input_mocap], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model_1 = model()
# model_1.summary()

In [9]:
# #speech_to_npy
# model1 = model()
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
#                               restore_best_weights=True)
# hist1 = model1.fit([x_train_text[:split],x_train_mocap[:split]], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate([x_train_text[split:],x_train_mocap[split:]], vad[split:].T.tolist())
# print(eval_metrik1)

# # make prediction
# predict = model1.predict([x_train_text[6290:],x_train_mocap[6290:]], batch_size=32)
# np.save('text_mocapGLOVE_npy7869', 
#          np.array(predict).reshape(3, 3749).T)


In [10]:
# main function for for LOSO
def main(time):
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [x_train_text[:7869],x_train_mocap[:7869]],
                      vad[:7869].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [x_train_text[7869:],x_train_mocap[7869:]], vad[7869:].T.tolist())
    print(metrik[-3:])
    predict = model_1.predict([x_train_text[6290:],x_train_mocap[6290:]], batch_size=32)
    np.save('npy7869/text_mocapGLOVE_7869npy'+str(time)+'.npy', 
         np.array(predict).reshape(3, 3749).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('npy7869/tm_glove_7869-batch-1.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 86s 14ms/step - loss: 1.9730 - v_loss: 0.6508 - a_loss: 0.6595 - d_loss: 0.6618 - v_ccc: 0.3487 - a_ccc: 0.3402 - d_ccc: 0.3381 - val_loss: 2.0087 - val_v_loss: 0.6969 - val_a_loss: 0.6508 - val_d_loss: 0.6844 - val_v_ccc: 0.3080 - val_a_ccc: 0.3581 - val_d_ccc: 0.3252
Epoch 2/50
6295/6295 [==============================] - 81s 13ms/step - loss: 1.4486 - v_loss: 0.3842 - a_loss: 0.5301 - d_loss: 0.5336 - v_ccc: 0.6155 - a_ccc: 0.4697 - d_ccc: 0.4662 - val_loss: 2.1392 - val_v_loss: 0.7301 - val_a_loss: 0.6909 - val_d_loss: 0.7349 - val_v_ccc: 0.2737 - val_a_ccc: 0.3162 - val_d_ccc: 0.2710
Epoch 3/50
6295/6295 [==============================] - 81s 13ms/step - loss: 1.2449 - v_loss: 0.3019 - a_loss: 0.4732 - d_loss: 0.4698 - v_ccc: 0.6979 - a_ccc: 0.5268 - d_ccc: 0.5304 - val_loss: 1.8654 - val_v_loss: 0

Epoch 3/50
6295/6295 [==============================] - 88s 14ms/step - loss: 1.2910 - v_loss: 0.3345 - a_loss: 0.4886 - d_loss: 0.4686 - v_ccc: 0.6655 - a_ccc: 0.5116 - d_ccc: 0.5319 - val_loss: 1.8683 - val_v_loss: 0.6041 - val_a_loss: 0.6350 - val_d_loss: 0.6466 - val_v_ccc: 0.3958 - val_a_ccc: 0.3734 - val_d_ccc: 0.3625
Epoch 4/50
6295/6295 [==============================] - 88s 14ms/step - loss: 1.1179 - v_loss: 0.2732 - a_loss: 0.4282 - d_loss: 0.4163 - v_ccc: 0.7267 - a_ccc: 0.5716 - d_ccc: 0.5837 - val_loss: 1.8704 - val_v_loss: 0.5753 - val_a_loss: 0.6226 - val_d_loss: 0.6865 - val_v_ccc: 0.4220 - val_a_ccc: 0.3856 - val_d_ccc: 0.3220
Epoch 5/50
6295/6295 [==============================] - 88s 14ms/step - loss: 0.9895 - v_loss: 0.2336 - a_loss: 0.3857 - d_loss: 0.3701 - v_ccc: 0.7662 - a_ccc: 0.6141 - d_ccc: 0.6302 - val_loss: 1.9020 - val_v_loss: 0.6179 - val_a_loss: 0.6472 - val_d_loss: 0.6614 - val_v_ccc: 0.3884 - val_a_ccc: 0.3611 - val_d_ccc: 0.3485
Epoch 6/50
6295/6295 [

Epoch 9/50
6295/6295 [==============================] - 87s 14ms/step - loss: 0.6945 - v_loss: 0.1637 - a_loss: 0.2711 - d_loss: 0.2597 - v_ccc: 0.8364 - a_ccc: 0.7288 - d_ccc: 0.7403 - val_loss: 1.8371 - val_v_loss: 0.5629 - val_a_loss: 0.6393 - val_d_loss: 0.6510 - val_v_ccc: 0.4359 - val_a_ccc: 0.3681 - val_d_ccc: 0.3589
Epoch 10/50
6295/6295 [==============================] - 88s 14ms/step - loss: 0.6428 - v_loss: 0.1442 - a_loss: 0.2584 - d_loss: 0.2408 - v_ccc: 0.8558 - a_ccc: 0.7420 - d_ccc: 0.7594 - val_loss: 1.8030 - val_v_loss: 0.5437 - val_a_loss: 0.6209 - val_d_loss: 0.6498 - val_v_ccc: 0.4513 - val_a_ccc: 0.3872 - val_d_ccc: 0.3585
Epoch 11/50
6295/6295 [==============================] - 87s 14ms/step - loss: 0.5827 - v_loss: 0.1375 - a_loss: 0.2309 - d_loss: 0.2143 - v_ccc: 0.8624 - a_ccc: 0.7692 - d_ccc: 0.7857 - val_loss: 1.8295 - val_v_loss: 0.5702 - val_a_loss: 0.6331 - val_d_loss: 0.6441 - val_v_ccc: 0.4279 - val_a_ccc: 0.3759 - val_d_ccc: 0.3667
Epoch 12/50
6295/629

Epoch 7/50
6295/6295 [==============================] - 86s 14ms/step - loss: 0.8300 - v_loss: 0.1916 - a_loss: 0.3265 - d_loss: 0.3122 - v_ccc: 0.8085 - a_ccc: 0.6734 - d_ccc: 0.6882 - val_loss: 1.8141 - val_v_loss: 0.5746 - val_a_loss: 0.6200 - val_d_loss: 0.6383 - val_v_ccc: 0.4248 - val_a_ccc: 0.3888 - val_d_ccc: 0.3722
Epoch 8/50
6295/6295 [==============================] - 86s 14ms/step - loss: 0.7522 - v_loss: 0.1774 - a_loss: 0.2966 - d_loss: 0.2790 - v_ccc: 0.8227 - a_ccc: 0.7038 - d_ccc: 0.7212 - val_loss: 1.7708 - val_v_loss: 0.5382 - val_a_loss: 0.6013 - val_d_loss: 0.6519 - val_v_ccc: 0.4629 - val_a_ccc: 0.4075 - val_d_ccc: 0.3589
Epoch 9/50
6295/6295 [==============================] - 87s 14ms/step - loss: 0.6876 - v_loss: 0.1620 - a_loss: 0.2702 - d_loss: 0.2554 - v_ccc: 0.8380 - a_ccc: 0.7298 - d_ccc: 0.7446 - val_loss: 1.7746 - val_v_loss: 0.5393 - val_a_loss: 0.6170 - val_d_loss: 0.6388 - val_v_ccc: 0.4599 - val_a_ccc: 0.3929 - val_d_ccc: 0.3726
Epoch 10/50
6295/6295 

Epoch 14/50
6295/6295 [==============================] - 81s 13ms/step - loss: 0.4518 - v_loss: 0.1109 - a_loss: 0.1741 - d_loss: 0.1669 - v_ccc: 0.8892 - a_ccc: 0.8259 - d_ccc: 0.8331 - val_loss: 1.7821 - val_v_loss: 0.5154 - val_a_loss: 0.6357 - val_d_loss: 0.6579 - val_v_ccc: 0.4917 - val_a_ccc: 0.3721 - val_d_ccc: 0.3542
Epoch 15/50
6295/6295 [==============================] - 82s 13ms/step - loss: 0.4154 - v_loss: 0.0991 - a_loss: 0.1629 - d_loss: 0.1532 - v_ccc: 0.9009 - a_ccc: 0.8370 - d_ccc: 0.8467 - val_loss: 1.7991 - val_v_loss: 0.5331 - val_a_loss: 0.6461 - val_d_loss: 0.6390 - val_v_ccc: 0.4712 - val_a_ccc: 0.3599 - val_d_ccc: 0.3698
Epoch 16/50
6295/6295 [==============================] - 81s 13ms/step - loss: 0.3933 - v_loss: 0.0950 - a_loss: 0.1548 - d_loss: 0.1437 - v_ccc: 0.9050 - a_ccc: 0.8453 - d_ccc: 0.8563 - val_loss: 1.8032 - val_v_loss: 0.5152 - val_a_loss: 0.6518 - val_d_loss: 0.6597 - val_v_ccc: 0.4906 - val_a_ccc: 0.3566 - val_d_ccc: 0.3495
Epoch 17/50
6295/62

...

Epoch 11/50
6295/6295 [==============================] - 82s 13ms/step - loss: 0.5658 - v_loss: 0.1351 - a_loss: 0.2222 - d_loss: 0.2087 - v_ccc: 0.8650 - a_ccc: 0.7777 - d_ccc: 0.7915 - val_loss: 1.7873 - val_v_loss: 0.5408 - val_a_loss: 0.6383 - val_d_loss: 0.6281 - val_v_ccc: 0.4608 - val_a_ccc: 0.3702 - val_d_ccc: 0.3817
Epoch 12/50
6295/6295 [==============================] - 82s 13ms/step - loss: 0.5170 - v_loss: 0.1230 - a_loss: 0.2033 - d_loss: 0.1908 - v_ccc: 0.8769 - a_ccc: 0.7968 - d_ccc: 0.8093 - val_loss: 1.8149 - val_v_loss: 0.5359 - val_a_loss: 0.6516 - val_d_loss: 0.6449 - val_v_ccc: 0.4647 - val_a_ccc: 0.3553 - val_d_ccc: 0.3651
Epoch 13/50
6295/6295 [==============================] - 82s 13ms/step - loss: 0.4770 - v_loss: 0.1145 - a_loss: 0.1912 - d_loss: 0.1714 - v_ccc: 0.8855 - a_ccc: 0.8089 - d_ccc: 0.8286 - val_loss: 1.8551 - val_v_loss: 0.5548 - val_a_loss: 0.6606 - val_d_loss: 0.6598 - val_v_ccc: 0.4491 - val_a_ccc: 0.3462 - val_d_ccc: 0.3495
Epoch 14/50
6295/62

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 81s 13ms/step - loss: 0.8801 - v_loss: 0.2001 - a_loss: 0.3486 - d_loss: 0.3319 - v_ccc: 0.8002 - a_ccc: 0.6516 - d_ccc: 0.6681 - val_loss: 1.8625 - val_v_loss: 0.5968 - val_a_loss: 0.6077 - val_d_loss: 0.6725 - val_v_ccc: 0.4013 - val_a_ccc: 0.4011 - val_d_ccc: 0.3352
Epoch 7/50
6295/6295 [==============================] - 81s 13ms/step - loss: 0.8079 - v_loss: 0.1887 - a_loss: 0.3134 - d_loss: 0.3056 - v_ccc: 0.8112 - a_ccc: 0.6865 - d_ccc: 0.6944 - val_loss: 1.8209 - val_v_loss: 0.5355 - val_a_loss: 0.6385 - val_d_loss: 0.6622 - val_v_ccc: 0.4612 - val_a_ccc: 0.3703 - val_d_ccc: 0.3477
Epoch 8/50
6295/6295 [==============================] - 82s 13ms/step - loss: 0.7285 - v_loss: 0.1655 - a_loss: 0.2869 - d_loss: 0.2764 - v_ccc: 0.8346 - a_ccc: 0.7132 - d_ccc: 0.7237 - val_loss: 1.7816 - val_v_loss: 0.5419 - val_a_loss: 0.6078 - val_d_loss: 0.6499 - val_v_ccc: 0.4573 - val_a_ccc: 0.4016 - val_d_ccc: 0.3595
Epoch 9/50
6295/6295 [===========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 83s 13ms/step - loss: 0.7520 - v_loss: 0.1712 - a_loss: 0.2939 - d_loss: 0.2874 - v_ccc: 0.8288 - a_ccc: 0.7064 - d_ccc: 0.7128 - val_loss: 1.8300 - val_v_loss: 0.5265 - val_a_loss: 0.6422 - val_d_loss: 0.6789 - val_v_ccc: 0.4717 - val_a_ccc: 0.3662 - val_d_ccc: 0.3322
Epoch 9/50
6295/6295 [==============================] - 83s 13ms/step - loss: 0.6856 - v_loss: 0.1557 - a_loss: 0.2689 - d_loss: 0.2611 - v_ccc: 0.8443 - a_ccc: 0.7312 - d_ccc: 0.7389 - val_loss: 1.8019 - val_v_loss: 0.5295 - val_a_loss: 0.6254 - val_d_loss: 0.6647 - val_v_ccc: 0.4707 - val_a_ccc: 0.3822 - val_d_ccc: 0.3451
Epoch 10/50
6295/6295 [==============================] - 83s 13ms/step - loss: 0.6244 - v_loss: 0.1455 - a_loss: 0.2437 - d_loss: 0.2351 - v_ccc: 0.8544 - a_ccc: 0.7563 - d_ccc: 0.7649 - val_loss: 1.7992 - val_v_loss: 0.5211 - val_a_loss: 0.6400 - val_d_loss: 0.6607 - val_v_ccc: 0.4800 - val_a_ccc: 0.3690 - val_d_ccc: 0.3518
Epoch 11/50
6295/6295 [=========